In [1]:
from graphviz import Digraph

# Initialize Graphviz Digraph
dot = Digraph(comment='Flask Route Diagram', format='png')

# Add nodes for routes
routes = [
    ('home', 'Home Page (/home)'),
    ('login', 'Login Page (/login)'),
    ('logout', 'Logout (/logout)'),
    ('signup', 'Signup Page (/signup)'),
    ('dashboard', 'Dashboard (/dashboard)'),
    ('manage_flights', 'Manage Flights (/manage_flights)'),
    ('view_flights', 'View Flights (/view_flights)'),
    ('track_spending', 'Track Spending (/track_spending)'),
    ('create_flight', 'Create Flight (/create_flight)'),
    ('grant_permissions', 'Grant Permissions (/grant_permissions)'),
    ('update_booking_agent_airline', 'Update Booking Agent Airline (/update_booking_agent_airline)'),
    ('view_commission', 'View Commission (/view_commission)'),
    ('view_top_customers', 'View Top Customers (/view_top_customers)')
]

for route, label in routes:
    dot.node(route, label)

# Define connections
connections = [
    ('home', 'login'),
    ('login', 'dashboard'),
    ('dashboard', 'logout'),
    ('dashboard', 'manage_flights'),
    ('dashboard', 'view_flights'),
    ('dashboard', 'track_spending'),
    ('dashboard', 'create_flight'),
    ('dashboard', 'grant_permissions'),
    ('grant_permissions', 'update_booking_agent_airline'),
    ('dashboard', 'view_commission'),
    ('dashboard', 'view_top_customers'),
]

for src, dest in connections:
    dot.edge(src, dest)

# Render the diagram
dot.render('flask_route_diagram', view=True)


'flask_route_diagram.png'